In [63]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
option = webdriver.ChromeOptions()
option.add_argument(' - incognito')

In [16]:
driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')

driver.get('https://www.carvana.com/cars?page=1')

driver.quit()

In [20]:
page = requests.get("http://dataquestio.github.io/web-scraping-pages/simple.html")
print(page.content)

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'


In [22]:
soup= BeautifulSoup(page.content,'html.parser')

In [23]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [24]:
list(soup.children)

['html', '\n', <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [25]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [31]:
html=list(soup.children)[2]

In [35]:
body=list(html.children)[3]

In [36]:
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [37]:
page = requests.get("http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

In [40]:
page= requests.get('https://forecast.weather.gov/MapClick.php?lat=39.9552&lon=-82.8055#.Xl_l26hKi70')
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id='seven-day-forecast')
forecast_items = seven_day.find_all(class_='tombstone-container')
tonight=forecast_items[1]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Mostly clear, with a low around 28. Northwest wind 5 to 10 mph becoming light and variable  after midnight. " class="forecast-icon" src="newimages/medium/nfew.png" title="Tonight: Mostly clear, with a low around 28. Northwest wind 5 to 10 mph becoming light and variable  after midnight. "/>
 </p>
 <p class="short-desc">
  Mostly Clear
 </p>
 <p class="temp temp-low">
  Low: 28 °F
 </p>
</div>


In [41]:
period = tonight.find(class_='period-name').get_text()
short_desc = tonight.find(class_='short-desc').get_text()
temp = tonight.find(class_='temp').get_text()
print(period)
print(short_desc)
print(temp)

Tonight
Mostly Clear
Low: 28 °F


In [45]:
img=tonight.find('img')
desc=img['title']
desc

'Tonight: Mostly clear, with a low around 28. Northwest wind 5 to 10 mph becoming light and variable  after midnight. '

In [48]:
period_tags = seven_day.select('.tombstone-container .period-name')
periods= [pt.get_text() for pt in period_tags]
periods

['ThisAfternoon',
 'Tonight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight',
 'Sunday']

In [61]:
short_descs = [sd.get_text() for sd in seven_day.select('.tombstone-container .short-desc')]
temps = [t.get_text() for t in seven_day.select('.tombstone-container .temp')]
descs = [d['title'] for d in seven_day.select('.tombstone-container img')]

In [62]:
descs

['This Afternoon: Showers likely, mainly between 1pm and 3pm.  Partly sunny, with a high near 54. Southwest wind 14 to 16 mph, with gusts as high as 26 mph.  Chance of precipitation is 60%. New precipitation amounts of less than a tenth of an inch possible. ',
 'Tonight: Mostly clear, with a low around 28. Northwest wind 5 to 10 mph becoming light and variable  after midnight. ',
 'Thursday: Sunny, with a high near 50. Light and variable wind becoming south 6 to 11 mph in the morning. ',
 'Thursday Night: A chance of rain showers, mixing with snow after 1am, then gradually ending.  Mostly cloudy, with a low around 33. Southwest wind 5 to 10 mph becoming west after midnight.  Chance of precipitation is 50%. Little or no snow accumulation expected. ',
 'Friday: A slight chance of snow showers before 9am, then a chance of rain and snow showers between 9am and 1pm, then a chance of rain showers after 1pm.  Mostly cloudy, with a high near 40. West wind 11 to 18 mph, with gusts as high as 30

In [64]:
weather = pd.DataFrame({'period':periods,'short_desc':short_descs,'temp':temps,'desc':descs})
weather

,period,short_desc,temp,desc
0,ThisAfternoon,ShowersLikely,High: 54 °F,"This Afternoon: Showers likely, mainly between..."
1,Tonight,Mostly Clear,Low: 28 °F,"Tonight: Mostly clear, with a low around 28. N..."
2,Thursday,Sunny,High: 50 °F,"Thursday: Sunny, with a high near 50. Light an..."
3,ThursdayNight,ChanceShowers thenChanceRain/Snow,Low: 33 °F,"Thursday Night: A chance of rain showers, mixi..."
4,Friday,ChanceRain/Snow,High: 40 °F,Friday: A slight chance of snow showers before...
5,FridayNight,Partly Cloudy,Low: 23 °F,"Friday Night: Partly cloudy, with a low around..."
6,Saturday,Sunny,High: 46 °F,"Saturday: Sunny, with a high near 46."
7,SaturdayNight,Clear,Low: 32 °F,"Saturday Night: Clear, with a low around 32."
8,Sunday,Sunny,High: 59 °F,"Sunday: Sunny, with a high near 59."


In [92]:
temp_nums = weather['temp'].str.extract('(\d+)',expand=False)
temp_nums

0    54
1    28
2    50
3    33
4    40
5    23
6    46
7    32
8    59
Name: temp, dtype: object

In [68]:
weather['temp_num']=temp_nums.astype('int')

In [70]:
weather['temp_num'].mean()

40.55555555555556